In [1]:
%load_ext autoreload
%autoreload 2

# LateFusionModel with semantics - ITLP dataset test

A module that implements a neural network algorithm for searching a database of places already visited by a vehicle for the most similar records using sequences of data from lidars and cameras.

In [2]:
import copy

from pathlib import Path
from time import time

import faiss
import numpy as np
import open3d as o3d
import open3d.core as o3c
import pandas as pd
import torch

from hydra.utils import instantiate
from omegaconf import OmegaConf
from scipy.spatial.transform import Rotation
from torch import Tensor
from torch.utils.data import DataLoader
from tqdm import tqdm

from opr.datasets.itlp import ITLPCampus
from opr.models.place_recognition.base import LateFusionModel
from opr.testing import test

2025-04-01 11:57:41.981 | WARNING  | opr.models.place_recognition.pointmamba:<module>:16 - The 'pointmamba' package is not installed. Please install it manually if neccessary.


In [3]:
import os

os.environ["DISPLAY"] = ":1"

torch.cuda.synchronize()

In [4]:
def pose_to_matrix(pose):
    """From the 6D poses in the [tx ty tz qx qy qz qw] format to 4x4 pose matrices."""
    position = pose[:3]
    orientation_quat = pose[3:]
    rotation = Rotation.from_quat(orientation_quat)
    pose_matrix = np.eye(4)
    pose_matrix[:3,:3] = rotation.as_matrix()
    pose_matrix[:3,3] = position
    return pose_matrix


# def compute_error(estimated_pose, gt_pose):
#     """For the 6D poses in the [tx ty tz qx qy qz qw] format."""
#     estimated_pose = pose_to_matrix(estimated_pose)
#     gt_pose = pose_to_matrix(gt_pose)
#     return compute_registration_error(estimated_pose, gt_pose)

def compute_error(estimated_pose, gt_pose):
    """For the 6D poses in the [tx ty tz qx qy qz qw] format."""
    estimated_pose = pose_to_matrix(estimated_pose)
    gt_pose = pose_to_matrix(gt_pose)
    error_pose = np.linalg.inv(estimated_pose) @ gt_pose
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = Rotation.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / np.pi
    angle_error = abs(90 - abs(angle_error-90))
    return angle_error, dist_error


def draw_pc(pc: Tensor, color: str = "blue"):
    pc_o3d = o3c.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(pc))
    pcd = o3d.t.geometry.PointCloud(pc_o3d)
    if color == "blue":
        c = [0.0, 0.0, 1.0]
    elif color == "red":
        c = [1.0, 0.0, 0.0]
    else:
        c = [0.0, 1.0, 0.0]
    pcd = pcd.paint_uniform_color(c)
    o3d.visualization.draw_geometries(
        [pcd.to_legacy()],
    )


def invert_rigid_transformation_matrix(T: np.ndarray) -> np.ndarray:
    """
    Inverts a 4x4 rigid body transformation matrix.

    Args:
        T (np.ndarray): A 4x4 rigid body transformation matrix.

    Returns:
        np.ndarray: The inverted 4x4 rigid body transformation matrix.
    """
    assert T.shape == (4, 4), "Input matrix must be 4x4."

    R = T[:3, :3]
    t = T[:3, 3]

    R_inv = R.T
    t_inv = -R.T @ t

    T_inv = np.eye(4)
    T_inv[:3, :3] = R_inv
    T_inv[:3, 3] = t_inv

    return T_inv


def draw_pc_pair(
    pc_blue: Tensor, pc_blue_pose: np.ndarray | Tensor, pc_red: Tensor, pc_red_pose: np.ndarray | Tensor
):
    pc_blue_o3d = o3c.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(copy.deepcopy(pc_blue)))
    pc_red_o3d = o3c.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(copy.deepcopy(pc_red)))

    blue_pcd = o3d.t.geometry.PointCloud(pc_blue_o3d)
    blue_pcd_tmp = copy.deepcopy(blue_pcd)

    red_pcd = o3d.t.geometry.PointCloud(pc_red_o3d)
    red_pcd_tmp = copy.deepcopy(red_pcd)

    blue_pcd_tmp.voxel_down_sample(voxel_size=0.3)
    # blue_pcd_tmp.transform(pose_to_matrix(pc_blue_pose))
    blue_pcd_tmp = blue_pcd_tmp.paint_uniform_color([0.0, 0.0, 1.0])

    red_pcd_tmp.voxel_down_sample(voxel_size=0.3)
    red_pcd_tmp.transform(pose_to_matrix(pc_red_pose))
    red_pcd_tmp.transform(invert_rigid_transformation_matrix(pose_to_matrix(pc_blue_pose)))
    red_pcd_tmp = red_pcd_tmp.paint_uniform_color([1.0, 0.0, 0.0])
    o3d.visualization.draw_geometries(
        [blue_pcd_tmp.to_legacy(), red_pcd_tmp.to_legacy()],
    )


You can **download the dataset**:

- Kaggle:
  - [ITLP Campus Outdoor](https://www.kaggle.com/datasets/alexandermelekhin/itlp-campus-outdoor)
- Hugging Face:
  - [ITLP Campus Outdoor](https://huggingface.co/datasets/OPR-Project/ITLP-Campus-Outdoor)

To **download the model weights**, run the following command:

```bash
# place recognition weights
wget -O ../../weights/place_recognition/multi-image_multi-semantic_lidar_late-fusion_nclt.pth https://huggingface.co/OPR-Project/PlaceRecognition-NCLT/resolve/main/multi-image_multi-semantic_lidar_late-fusion_nclt.pth
wget -O ../../weights/place_recognition/multi-image_multi-semantic_lidar_late-fusion_itlp-finetune.pth https://huggingface.co/OPR-Project/PlaceRecognition-NCLT/resolve/main/multi-image_multi-semantic_lidar_late-fusion_itlp-finetune.pth

# registration weights
wget -O ../../weights/registration/hregnet_light_feats_nuscenes.pth https://huggingface.co/OPR-Project/Registration-nuScenes/resolve/main/hregnet_light_feats_nuscenes.pth
```


In [ ]:
# place recognition weights
!wget -O ../../weights/place_recognition/multi-image_multi-semantic_lidar_late-fusion_nclt.pth https://huggingface.co/OPR-Project/PlaceRecognition-NCLT/resolve/main/multi-image_multi-semantic_lidar_late-fusion_nclt.pth
!wget -O ../../weights/place_recognition/multi-image_multi-semantic_lidar_late-fusion_itlp-finetune.pth https://huggingface.co/OPR-Project/PlaceRecognition-NCLT/resolve/main/multi-image_multi-semantic_lidar_late-fusion_itlp-finetune.pth

# registration weights
!wget -O ../../weights/registration/hregnet_light_feats_nuscenes.pth https://huggingface.co/OPR-Project/Registration-nuScenes/resolve/main/hregnet_light_feats_nuscenes.pth

In [5]:
DATASET_ROOT = "/home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor"

TRACK_LIST = sorted(
    [str(subdir.name) for subdir in Path(DATASET_ROOT).iterdir() if subdir.is_dir() and not subdir.name.startswith(".")]
)
TRACK_LIST = TRACK_LIST[5:]

print("Test track list:")
print(TRACK_LIST)

Test track list:
['05_2023-08-15-day', '06_2023-08-18-night', '07_2023-10-04-day', '08_2023-10-11-night']


# Original version

In [6]:
SENSOR_SUITE = ["front_cam", "back_cam", "lidar"]

BATCH_SIZE = 4
NUM_WORKERS = 4
DEVICE = "cuda"

PR_MODEL_CONFIG_PATH = "../../configs/model/place_recognition/multi-image_multi-semantic_lidar_late-fusion.yaml"
PR_WEIGHTS_PATH = "../../weights/place_recognition/multi-image_multi-semantic_lidar_late-fusion_nclt.pth"

## Init models

In [7]:
pr_model_config = OmegaConf.load(PR_MODEL_CONFIG_PATH)
pr_model = instantiate(pr_model_config)
pr_model.load_state_dict(torch.load(PR_WEIGHTS_PATH))
pr_model = pr_model.to(DEVICE)
pr_model.eval();


## Create dataloader

In [8]:
dataset = ITLPCampus(
    dataset_root=DATASET_ROOT,
    subset="test",
    csv_file="test.csv",
    sensors=SENSOR_SUITE,
    load_semantics=True,
)
dataset.dataset_df = dataset.dataset_df[dataset.dataset_df["track"].isin(TRACK_LIST)]
dataset.dataset_df.reset_index(inplace=True)

no_masks = []
no_masks_filenames = []

for index, row in dataset.dataset_df.iterrows():
    mask_path = f"{DATASET_ROOT}/{row['track']}/masks/back_cam/{row['back_cam_ts']}.png"
    filename = f"{row['back_cam_ts']}.png"
    if not Path(mask_path).exists():
        no_masks.append(index)
        no_masks_filenames.append(filename)

dataset.dataset_df.drop(no_masks, inplace=True)
dataset.dataset_df.reset_index(inplace=True)

dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=dataset.collate_fn,
)


In [9]:
len(no_masks)

0

## Test

In [10]:
ar_at_n, ar_at_1per, mean_top1_distance = test(model=pr_model, dataloader=dataloader)

## Result

In [12]:
print(f"Mean top-1 Euclidean distance: {mean_top1_distance:.4f}")

Mean top-1 Euclidean distance: 1.3713


# Improved version

In [13]:
SENSOR_SUITE = ["front_cam", "back_cam", "lidar"]

BATCH_SIZE = 4
NUM_WORKERS = 4
DEVICE = "cuda"

PR_MODEL_CONFIG_PATH = "../../configs/model/place_recognition/multi-image_multi-semantic_lidar_late-fusion.yaml"
PR_WEIGHTS_PATH = "../../weights/place_recognition/multi-image_multi-semantic_lidar_late-fusion_itlp-finetune.pth"

REGISTRATION_MODEL_CONFIG_PATH = "../../configs/model/registration/hregnet_light_feats.yaml"
REGISTRATION_WEIGHTS_PATH = "../../weights/registration/hregnet_light_feats_nuscenes.pth"

## Init models

In [14]:
pr_model_config = OmegaConf.load(PR_MODEL_CONFIG_PATH)
pr_model = instantiate(pr_model_config)
pr_model.load_state_dict(torch.load(PR_WEIGHTS_PATH))
pr_model = pr_model.to(DEVICE)
pr_model.eval();

## Create dataloader

In [16]:
dataset = ITLPCampus(
    dataset_root=DATASET_ROOT,
    subset="test",
    csv_file="test.csv",
    sensors=SENSOR_SUITE,
    load_semantics=True,
)
dataset.dataset_df = dataset.dataset_df[dataset.dataset_df["track"].isin(TRACK_LIST)]
dataset.dataset_df.reset_index(inplace=True)

no_masks = []
no_masks_filenames = []

for index, row in dataset.dataset_df.iterrows():
    mask_path = f"{DATASET_ROOT}/{row['track']}/masks/back_cam/{row['back_cam_ts']}.png"
    filename = f"{row['back_cam_ts']}.png"
    if not Path(mask_path).exists():
        no_masks.append(index)
        no_masks_filenames.append(filename)

dataset.dataset_df.drop(no_masks, inplace=True)
dataset.dataset_df.reset_index(inplace=True)

dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=dataset.collate_fn,
)


## Test

In [17]:
ar_at_n, ar_at_1per, mean_top1_distance = test(model=pr_model, dataloader=dataloader)

## Result

In [18]:
print(f"Mean top-1 Euclidean distance: {mean_top1_distance:.4f}")

Mean top-1 Euclidean distance: 1.2725
